# Price Predictor Linear Regression Model

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## Load in data

In [2]:
ada_df = pd.read_csv(Path('../cardano_2017-10-1_2022-4-27.csv'))
ada_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,Apr-26-2022,0.897602,0.900668,0.819120,0.824140,1.311330e+09,2.730421e+10
1,Apr-25-2022,0.885730,0.899764,0.830142,0.898860,1.175064e+09,2.721793e+10
2,Apr-24-2022,0.887093,0.897949,0.880130,0.885290,5.521224e+08,2.800958e+10
3,Apr-23-2022,0.906163,0.910934,0.883990,0.888474,7.555170e+08,2.826530e+10
4,Apr-22-2022,0.909063,0.923691,0.894321,0.907236,1.201371e+09,2.867515e+10


In [3]:
ada_df.shape

(1668, 7)

In [4]:
ada_df.dtypes

Date           object
Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
Market Cap    float64
dtype: object

## Transform datatypes